<a href="https://colab.research.google.com/github/brettmorrisonAO5ANNEX/Super-Resolution-Implementation/blob/main/SRCNN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

##Data Preprocessing

In [ ]:
def load_image(image_path):
    return Image.open(image_path)

def preprocess_image(image, scale_factor=3):
    image = image.convert('RGB')
    image = np.array(image).astype(np.float32) / 255.0
    input_image = image[::scale_factor, ::scale_factor, :]  # Downsample by the scale factor
    return input_image, image

# Example to load and preprocess images
image_paths = ["path_to_image1.jpg", "path_to_image2.jpg"]  # Replace with actual paths
input_images = []
target_images = []

for path in image_paths:
    input_img, target_img = preprocess_image(load_image(path))
    input_images.append(input_img)
    target_images.append(target_img)

input_images = np.array(input_images)
target_images = np.array(target_images)

# Split the data
x_train, x_test, y_train, y_test = train_test_split(input_images, target_images, test_size=0.2, random_state=42)

##Creating The Model

In [ ]:
def build_srcnn():
    model = Sequential()
    # First convolutional layer
    model.add(Conv2D(filters=64, kernel_size=(9, 9), activation='relu', padding='same', input_shape=(None, None, 3)))
    # Second convolutional layer
    model.add(Conv2D(filters=32, kernel_size=(1, 1), activation='relu', padding='same'))
    # Third convolutional layer
    model.add(Conv2D(filters=3, kernel_size=(5, 5), activation='linear', padding='same'))
    return model

srcnn = build_srcnn()
srcnn.compile(optimizer=Adam(learning_rate=0.0003), loss='mean_squared_error', metrics=['mean_squared_error'])

##Training & Evaluation

In [ ]:
history = srcnn.fit(x_train, y_train, epochs=100, batch_size=16, validation_data=(x_test, y_test))

In [ ]:
loss, mse = srcnn.evaluate(x_test, y_test)
print(f"Test MSE: {mse}")

# Visualize the results
def show_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Predict and visualize
sample_input = x_test[0]
predicted_image = srcnn.predict(sample_input[np.newaxis, ...])
show_image(sample_input)
show_image(predicted_image[0])
show_image(y_test[0])